In [ ]:
# airports
class Airports:
    def __init__(self,n):
        self._n = n + 1
        self._neighbors = [[] for _ in range(self._n)]

    def add_link(self,a,b):
        self._neighbors[a].append(b)

    def _dfs(self, node, visited):
        if visited[node]:
            return
        visited[node] = True
        for i in self._neighbors[node]:
            self._dfs(i, visited)

    def check(self):
        res = True
        for i in range(1, self._n):
            visited = [False]*self._n
            self._dfs(i, visited)
            res = res and False not in visited[1:]
        return res


if __name__ == "__main__":
    a = Airports(5)
    a.add_link(1,2)
    a.add_link(2,3)
    a.add_link(1,3)
    a.add_link(4,5)
    print(a.check()) # False
    a.add_link(3,5)
    a.add_link(1,4)
    print(a.check()) # False
    a.add_link(5,1)
    print(a.check()) # True

In [ ]:
# courseplan
class CoursePlan:
    def __init__(self):
        self._reqs = {}
    
    def add_course(self,course):
        self._reqs[course] = []

    def add_requisite(self,course1,course2):
        self._reqs[course1].append(course2)

    def _dfs(self, course, visited, tsort):
        if visited[course] == 2:
            return
        if visited[course] == 1:
            self._cycle = True
            return
        visited[course] = 1
        for req in self._reqs[course]:
            self._dfs(req, visited, tsort)
        visited[course] = 2
        tsort.append(course)


    def find(self):
        tsort = []
        visited = dict.fromkeys(self._reqs, 0)
        self._cycle = False
        for course in self._reqs.keys():
            if visited[course] == 0:
                self._dfs(course, visited, tsort)
            if self._cycle:
                return None
        return list(reversed(tsort))

if __name__ == "__main__":
    c = CoursePlan()
    c.add_course("Ohpe")
    c.add_course("Ohja")
    c.add_course("Tira")
    c.add_course("Jym")
    c.add_requisite("Ohpe","Ohja")
    c.add_requisite("Ohja","Tira")
    c.add_requisite("Jym","Tira")
    print(c.find()) # [Ohpe,Jym,Ohja,Tira]
    c.add_requisite("Tira","Tira")
    print(c.find()) # None

In [ ]:
# cycles
class Cycles:
    class CycleError(Exception):
        pass

    def __init__(self,n):
        self._neighbours = [[] for _ in range(n+1)]

    def add_edge(self,a,b):
        self._neighbours[a].append(b)

    def _dfs(self, n, visited):
        if visited[n] == 2:
            return
        if visited[n] == 1:
            # solmuun tultu syklin kautta
            raise self.CycleError
        visited[n] = 1
        for i in self._neighbours[n]:
            self._dfs(i, visited)
        visited[n] = 2

    def check(self):
        visited = [0]*len(self._neighbours)
        while True:
            try:
                i = visited.index(0)
            except ValueError:
                # kaikki solmut käsitelty löytämättä syklejä
                return False
            try:
                self._dfs(i, visited)
            except self.CycleError:
                return True

if __name__ == "__main__":
    c = Cycles(4)
    c.add_edge(1,2)
    c.add_edge(2,3)
    c.add_edge(1,3)
    c.add_edge(3,4)
    print(c.check()) # False
    c.add_edge(4,2)
    print(c.check()) # True

In [ ]:
# graphgame
class GraphGame:
    def __init__(self,n):
        self._n = n+1
        self._neighbors = [[] for _ in range(self._n)]
        self._wins = [None]*self._n

    def add_link(self,a,b):
        self._neighbors[a].append(b)
        # ratkaisu muuttunut
        self._wins = [None]*self._n

    def winning(self,x):
        if self._wins[x] != None:
            return self._wins[x]
        else:
            for i in self._neighbors[x]:
                if not self.winning(i):
                    self._wins[x] = True
                    return True
            self._wins[x] = False
            return False


if __name__ == "__main__":
    g = GraphGame(6)
    g.add_link(3,4)
    g.add_link(1,4)
    g.add_link(4,5)
    print(g.winning(3)) # False
    print(g.winning(1)) # False
    g.add_link(3,1)
    g.add_link(4,6)
    g.add_link(6,5)
    print(g.winning(3)) # True
    print(g.winning(1)) # False
    print(g.winning(2)) # False

    g = GraphGame(10)
    g.add_link(4,1)
    g.add_link(7,6)
    print(g.winning(3))
    g.add_link(5,9)
    print(g.winning(4))
    g.add_link(8,10)
    g.add_link(1,6)
    g.add_link(10,6)
    print(g.winning(10))
    g.add_link(4,1)
    g.add_link(9,4)
    print(g.winning(2))
    g.add_link(2,4)
    g.add_link(4,1)
    g.add_link(5,8)
    print(g.winning(5))
    print(g.winning(4)) # False
    g.add_link(1,6)
    print(g.winning(8))
    print(g.winning(2))

In [1]:
# longpath
class LongPath:
    def __init__(self,n):
        self._n = n+1
        self._neighbors = [[] for _ in range(self._n)]
        self._predecessors = [[] for _ in range(self._n)]

    def add_edge(self,a,b):
        if b > a:
            self._neighbors[a].append(b)
            self._predecessors[b].append(a)
        else:
            self._neighbors[b].append(a)
            self._predecessors[a].append(b)

    def _dfs(self, node, visited, sorted):
        if visited[node] == 2:
            return
        elif visited[node] == 1:
            # ei ole DAG
            raise ValueError
        else:
            visited[node] = 1
            for i in self._neighbors[node]:
                self._dfs(i, visited, sorted)
            visited[node] = 2
            sorted.append(node)

    def calculate(self):
        # https://www.mathcs.emory.edu/~cheung/Courses/171/Syllabus/11-Graph/Docs/longest-path-in-dag.pdf
        # 1. topologinen järjestys
        sorted = []
        visited = [0]*self._n
        for i in range(1, self._n):
            if visited[i] == 0:
                self._dfs(i, visited, sorted)
        sorted.reverse()
        
        # 2. lasketaan pisin polku jokaiseen solmuun
        distances = [0]*self._n
        for v in sorted:
            if len(self._predecessors[v]) > 0:
                distances[v] = max([distances[i] + 1 for i in self._predecessors[v]])
        
        # 3. palautetaan pisin kaikista poluista
        return max(distances)


if __name__ == "__main__":
    l = LongPath(4)
    l.add_edge(1,2)
    l.add_edge(1,3)
    l.add_edge(2,4)
    l.add_edge(3,4)
    print(l.calculate()) # 2
    l.add_edge(3,2)
    print(l.calculate()) # 3

2
3


In [ ]:
# paths
def polut(x):
    pit = [0]*(x+1)
    pit[1] = 1
    pit[2] = 1
    for i in range(3, x+1):
        pit[i] = pit[i-1] + pit[i-2]
    return pit[x]

print(polut(50))